## Импорт библиотек

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms

## Инициализация устройства

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Устройство:", device)

Устройство: cuda


## Загрузка и трансформация данных

In [8]:
data_dir = "compressed_dataset"
batch_size = 32

transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

dataset = datasets.ImageFolder(data_dir, transform=transform)

## Разделение на train/test

In [9]:
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size], generator=torch.Generator().manual_seed(42))

## Загрузчики данных

In [10]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## Определение архитектуры модели

In [5]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64, 500)
        self.fc2 = nn.Linear(500, 124)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = x.view(-1, 64)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


model = CNN().to(device)

## Обучение модели

In [13]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')
    
# Сохранение модели
torch.save(model.state_dict(), 'model.pth')

Epoch [1/10], Loss: 3.0163
Epoch [2/10], Loss: 2.7727
Epoch [3/10], Loss: 3.0085
Epoch [4/10], Loss: 2.4008
Epoch [5/10], Loss: 2.2546
Epoch [6/10], Loss: 3.1655
Epoch [7/10], Loss: 3.0756
Epoch [8/10], Loss: 2.3907
Epoch [9/10], Loss: 2.8154
Epoch [10/10], Loss: 2.5103


## Тестирование модели

In [ ]:
# Загрузка модели (Если требуется)
model = CNN()
model.load_state_dict(torch.load('model.pth'))

In [14]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the test images: {100 * correct / total}%')

Accuracy of the network on the test images: 33.64%


## Проверка на реальных примерах

In [11]:
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms

model = CNN()
model.load_state_dict(torch.load('model.pth'))
model.to(device)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Устройство:", device)
with open('dictionary.json', 'r') as f:
    loaded_dict = json.load(f)
    
def predict_image(image_path):
    transform = transforms.Compose([
        transforms.Resize((64, 64)),
        transforms.ToTensor(),
    ])
    image = Image.open(image_path)
    if image.mode == 'RGBA':
        image = image.convert('RGB')
    image = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output, 1)
        return predicted.item()

Устройство: cuda


In [ ]:
image_path = "Exmaple.jpg"
predicted_class = predict_image(image_path)

def get_name_from_number(number):
    number_str = str(number)
    name = file_dict.get(number_str)
    return name

print("Predicted class:", get_name_from_number(predicted_class))